In [1]:
!nvidia-smi

Tue Jul 30 01:21:34 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 462.62       Driver Version: 462.62       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 305... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   40C    P3    13W /  N/A |    105MiB /  4096MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr

In [3]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

  Using cached accelerate-0.33.0-py3-none-any.whl.metadata (18 kB)
  Using cached numpy-1.26.4-cp310-cp310-win_amd64.whl.metadata (61 kB)
Using cached accelerate-0.33.0-py3-none-any.whl (315 kB)
Using cached numpy-1.26.4-cp310-cp310-win_amd64.whl (15.8 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.1
    Uninstalling numpy-2.0.1:
      Successfully uninstalled numpy-2.0.1
Found existing installation: transformers 4.43.3
Uninstalling transformers-4.43.3:
  Successfully uninstalled transformers-4.43.3
Found existing installation: accelerate 0.33.0
Uninstalling accelerate-0.33.0:
  Successfully uninstalled accelerate-0.33.0
^C


  Using cached transformers-4.43.3-py3-none-any.whl.metadata (43 kB)
  Using cached accelerate-0.33.0-py3-none-any.whl.metadata (18 kB)
Using cached transformers-4.43.3-py3-none-any.whl (9.4 MB)
Using cached accelerate-0.33.0-py3-none-any.whl (315 kB)


In [ ]:
from transformers import pipeline, set_seed
from datasets import load_dataset, load_from_disk,load_metric
import matplotlib.pyplot as plt
import pandas as pd
from transformers import AutoModelForSeq2SeqLM,AutoTokenizer
import nltk

from nltk.tokenize import sent_tokenize
from tqdm import tqdm
import torch

nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu' ## checking if gpu is avaliable or not
model_ckpt = 'google/pegasus-cnn_dailymail'
tokenizer = AutoTokenizer.from_pretrained(model_ckpt) ## pretrained tokenizer method , as taken by the model which is google pegasus, cnn dailymail
model=AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device) ##loading the model in the gpu , for faster processing


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [5]:
##loading the data into the machine(google colab provided) and then unzipping it
!wget https://github.com/entbappy/Branching-tutorial/raw/master/summarizer-data.zip
## unzipping this downloaded file
!unzip summarizer-data.zip


'which' is not recognized as an internal or external command,
operable program or batch file.


In [8]:
dataset_samsum

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [37]:
## this is the basic way to configur the dataset
dataset_samsum['train'][1]['summary']

'Olivia and Olivier are voting for liberals in this election. '

In [9]:
split_lengths=[len(dataset_samsum[i]) for i in  dataset_samsum]
print(f"features of the data:{dataset_samsum['train'].column_names}")
print(f"split_lengths : {split_lengths}")
print("dialogue one")
print(dataset_samsum['test'][1]['dialogue'])
print(dataset_samsum['test'][1]['summary'])

features of the data:['id', 'dialogue', 'summary']
split_lengths : [14732, 819, 818]
dialogue one
Eric: MACHINE!
Rob: That's so gr8!
Eric: I know! And shows how Americans see Russian ;)
Rob: And it's really funny!
Eric: I know! I especially like the train part!
Rob: Hahaha! No one talks to the machine like that!
Eric: Is this his only stand-up?
Rob: Idk. I'll check.
Eric: Sure.
Rob: Turns out no! There are some of his stand-ups on youtube.
Eric: Gr8! I'll watch them now!
Rob: Me too!
Eric: MACHINE!
Rob: MACHINE!
Eric: TTYL?
Rob: Sure :)
Eric and Rob are going to watch a stand-up on youtube.


In [10]:
##this helps us get tokens out of the input and output features
## creating vector embeddings 
def convert_examples_to_features(batch_example):
  input_encodings = tokenizer(batch_example['dialogue'],max_length=1024,truncation=True)

  with tokenizer.as_target_tokenizer():
    target_encodings = tokenizer(batch_example['summary'],max_length=1024,truncation=True)
  return{
      "input_ids" : input_encodings['input_ids'],
      "attention_mask" : input_encodings['attention_mask'],
      "labels" : target_encodings['input_ids']
  }

In [11]:
## passing this data into the tokenizer function and adds eavh returned value to the respective row
dataset_samsum_pt = dataset_samsum.map(convert_examples_to_features, batched = True)

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4144: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

In [38]:
dataset_samsum_pt

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 818
    })
})

In [12]:
## training , it helps us add padding and completing the features properly
from transformers import DataCollatorForSeq2Seq

seq2seq_data_collator=DataCollatorForSeq2Seq(tokenizer,model = model)

In [13]:

from transformers import TrainingArguments, Trainer
##arguments for training model where wehat we have is the basic output_directory , num of epochs, warmup steps , weight_decay
##gradient_accumulation_step
trainer_args = TrainingArguments(
    output_dir='pegasus-samsum', num_train_epochs=1, warmup_steps=500,
    per_device_train_batch_size=1, per_device_eval_batch_size=1,
    weight_decay=0.01, logging_steps=10,
    evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
    gradient_accumulation_steps=16
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [14]:
train=Trainer(model=model,args=trainer_args,tokenizer=tokenizer,data_collator=seq2seq_data_collator,train_dataset=dataset_samsum_pt['train'],eval_dataset=dataset_samsum_pt['validation'])

In [15]:
train.train()

Step,Training Loss,Validation Loss
500,1.659900,1.483296


Step,Training Loss,Validation Loss
500,1.659900,1.483296


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8, 'forced_eos_token_id': 1}


TrainOutput(global_step=920, training_loss=1.8251974468645842, metrics={'train_runtime': 2861.9059, 'train_samples_per_second': 5.148, 'train_steps_per_second': 0.321, 'total_flos': 5528248038285312.0, 'train_loss': 1.8251974468645842, 'epoch': 0.9991854466467553})

In [2]:
##evaluation
## we are checking how well the model is performing on unseen data
## a generating batch sized chunks for generating the batch size
def generate_batch_sized_chunks(list_of_elements,batch_size):
  for i in range(0,len(list_of_elements),batch_size):
    yield list_of_elements[i:i+batch_size]

##calculating the eval metrics in the form of rouge
def calculate_metric_on_test_ds(dataset,metric,model,tokenizer,batch_size=16,deivce=device,column_text='article',column_summary="highlights"):
    article_batches=list(generate_batch_sized_chunks(dataset['dialogue'],batch_size))
    target_batches=list(generate_batch_sized_chunks(dataset['summary'],batch_size))

    for article_batches,target_batches in tqdm(zip(article_batches,target_batches),total=len(article_batches)):
      inputs = tokenizer(article_batches,max_length=1024,truncation=True,padding='max_length',return_tensors='pt')
      summaries= model.generate(input_ids=inputs['input_ids'].to(device),attention_mask=inputs['attention_mask'].to(device),length_penalty=0.8,num_beams=8,max_length=128)
      decoded_summaries=[tokenizer.decode(s,skip_special_tokens=True,clean_up_tokenization_spaces=True) for s in summaries]
      decoded_summaries=[d.replace("<n>"," ") for d in decoded_summaries]
      metric.add_batch(predictions=decoded_summaries,references=target_batches)
    score=metric.compute()
    return score



NameError: name 'device' is not defined

In [ ]:
rouge_names=['rouge1','rouge2','rougeL','rougeLsum']
rouge_metric=load_metric('rouge')

<ipython-input-17-c19d9619cfa6>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge_metric=load_metric('rouge')


The repository for rouge contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/rouge.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


In [26]:
score=calculate_metric_on_test_ds(dataset_samsum['test'][0:10],rouge_metric,model,tokenizer,batch_size=2,column_text='dialogue',column_summary='summary')
rouge_dict=dict((rn,score[rn].mid.fmeasure) for rn in rouge_names)
pd.DataFrame(rouge_dict,index=[f'pegasus'])

100%|██████████| 5/5 [00:17<00:00,  3.56s/it]


,rouge1,rouge2,rougeL,rougeLsum
pegasus,0.429263,0.219829,0.363417,0.36485


In [18]:
model.save_pretrained("pegasus-samsum-model")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8, 'forced_eos_token_id': 1}


In [27]:
##saving the tokenizer as well
tokenizer.save_pretrained("tokenizer")

('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/spiece.model',
 'tokenizer/added_tokens.json',
 'tokenizer/tokenizer.json')

In [40]:
## we are now usng the now trianed model
gen_kwargs = {"length_penalty": 1, "num_beams":12, "max_length": 128}

sample_text=dataset_samsum['test'][0]['dialogue']
reference=dataset_samsum['test'][0]['summary']
## we are using pipeline functioning of transformers library to create the model
pipe=pipeline('summarization',model='pegasus-samsum-model',tokenizer=tokenizer)
print(sample_text)
print(pipe(sample_text, **gen_kwargs)[0]['summary_text'])

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Your max_length is set to 128, but your input_length is only 122. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=61)


Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye
Hannah is looking for Betty's number. Amanda can't find it. Larry called Betty last time they were at the park together. Hannah wants Amanda to text him.
